In [99]:
import os
import re
import json
from tqdm import tqdm
from utils.call_gpt import call_gpt, call_gpt_async

In [ ]:
your_api_key=" "
your_api_base=" "
your_model_name=" "

四大关系: refer_to, exclude, only_include, should_include

refer_to 条款引用
 - 源cu需要结合目标条款的信息作为补充, 才能完整解释. 注意: "某某角色/情况的认定参考某某法律法规文件/法条", "处于某某法律法规文件/法条规定的的特殊情形", "参考附件2", "具体要求见附件4"这种需要参考其他文件/法条来判断的, 属于refer_to; "后续处理应当按照《公司法》、中国证监会和本所的相关规定办理"这种外部的整个法律的遵守也属于refer_to. 
 - 如果是"应当遵守某某法条", "在某情况下不适用/免于遵守某某法条", "只需要遵守某某法条", 这些在本法律法规文件内部进行遵守/免于遵守的关系, 分别属于should_include, exclude和only_include, 而非refer_to
 - 公司章程也可以被refer_to
 - 这是在生成cu的函数时候起作用. Coding Agent需要去查找refer_to的目标条款
  
exclude 规则排除
 - 源cu成立时(源cu的主体subject符合, 且条件condition符合时), 使目标cu失效. 
 - 关键词: 不受前款限制, 免于遵守xxx, 
 - 这是在cu的函数执行后, 计算整个图的违规与否时起作用. 此时所有的cu已经有这些结果: [主体适用/不适用, 条件符合/不符合, 违规/不违规], 此时找到exclude关系, 对于exclude的source, 如果其主体适用, 条件符合, 那么其target的cu就标注为免除
  
only_include 仅适用
 - 源cu成立时(源cu的主体subject符合, 且条件condition符合时), 只需要考虑目标cu的情况, 而不再需要考虑本法律法规文件内的任何其他的cu. 
 - 这是在cu的函数执行后, 计算整个图的违规与否时起作用. 
  
should_include 强制纳入
 - 当cu_n_k中出现 {{"应当符合/遵循本指引第m, n, k条的要求", "应当参照/按照/参见本指引第m, n, k条处理",}} 时, 就是当前的cu_n_k对Law_x, ..., Law_z存在should_include关系. 
 - should_include 的特点是可以免除一部分目标cu的condition, 例如要约回购的cu要求主体遵循某个condition为竞价回购的cu的constrain, 此时应当直接检查constrain, 忽视condition的冲突. 
 - 只有明确声明应当遵循本法律法规内哪些法条的, 才是should_include关系. 指向目标不在本法律法规文件内部的, 以及未明确说明的应当遵循哪些法条的不予考虑. 注意: "某某情况的认定参考某某法律文件"的属于refer_to关系; "只需要考虑/遵守/按照某某要求办理"的属于only_include; 源cu成立时可以免除目标cu的属于exclude关系. 这些都不属于only_include关系. 
 - 这是在cu的函数执行后, 计算整个图的违规与否时起作用. 强制纳入有时候会出现condition为"要约回购"的案例也符合condition为"竞价回购"的cu的要求, 因此需要有针对性地生成新的强制纳入cu, 或者考虑免除一些condition. 这里需要更多的例子. 

# prompt模板

In [101]:
prompt_get_relation = """
# cu关系识别指令 **RELATION**

## 角色定位
你是一个资深法律条款引用分析专家，专注识别 cu (合规检查单元) 中的**RELATION**关系

## cu概念简述
cu（ComplianceUnit）是法律条文拆解出的最小合规单元，包含：
- cu_id: cu的编号, 通常为"cu_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东"）
- condition: 触发条件（如"减持股份"） 
- constraint: 约束内容（如"提前15日公告"）
- contextual_info: 补充说明（如价格计算方式）
我们采用cu判断案例的合规性时, 会先检查案例主体是否符合cu主体, 再检查案例中的条件是否符合cu中的条件, 当前两者都满足, 再检查案例中的主体的行为是否违反了cu中的约束. 只有主体, 条件和约束全部满足, 才会认为该案例在该cu上违规, 否则会判定该案例在该cu上不违规. 

## 核心任务
从给定的cu列表中识别 **RELATION** 关系，输出(source_id, **RELATION**, target_id)三元组. 
当target_id有多个时可以用列表承载, 例如(source_id, **RELATION**, [target_id_1, target_id_2])
当所给的cu之间或者cu与法律之间不存在**RELATION**时, 如实返回空值, 不需要自己杜撰或强行拼凑**RELATION**. cu间不存在关系是普遍的现象. 

## 注意事项
- 源cu需要结合目标条款才能完整解释. 当**RELATION**的目标为某个具体cu时候直接记录其编号cu_n_k. 当**RELATION**的目标为本法(或者"本指引")的第n条法条时, 编号为"Law_n". 
- 示例：
data:
[
    ...
]
relation: 
    ...


## 识别原则
1. 不修改cu内容，仅建立关联
2. 允许cu之间建立关系, 让cu与法条建立关系, 以及跨法条建立关系
3. 请遵循奥卡姆剃刀原则, 不要增加relation, 除非它是必要的.

## 输出格式
用<RELATIONS>标签包裹的 Python 列表, 列表内为一个个relation元组, 不要有任何注释等赘余内容. 下面是一个输出的样例: 
<RELATIONS>
[
("cu_n_i", "only_include", "cu_n_j"),
("Law_n", "only_include", "Law_m")
]
</RELATIONS>


接下来是等待你发掘关系的cu列表:
{cu_list}
"""

## refer_to

In [102]:
prompt_get_refer_to = """
# cu关系识别指令 refer_to (条款引用)

## 角色定位
你是一个资深法律条款引用分析专家，专注识别 cu (合规检查单元) 中的refer_to (条款引用)关系

## cu概念简述
cu (ComplianceUnit) 是法律条文拆解出的最小合规单元，包含：
- cu_id: cu的编号, 通常为"cu_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东") 
- condition: 触发条件（如"减持股份") 
- constraint: 约束内容（如"提前15日公告") 
- contextual_info: 补充说明（如价格计算方式）
我们采用cu判断案例的合规性时, 会先检查案例主体是否符合cu主体, 再检查案例中的条件是否符合cu中的条件, 当前两者都满足, 再检查案例中的主体的行为是否违反了cu中的约束. 只有主体, 条件和约束全部满足, 才会认为该案例在该cu上违规, 否则会判定该案例在该cu上不违规. 

## 核心任务
从给定的cu列表中识别 refer_to (条款引用) 关系，输出(source_id, refer_to, target_id)三元组. 
当target_id有多个时可以用列表承载, 例如(source_id, refer_to, [target_id_1, target_id_2])
当所给的cu之间或者cu与法律之间不存在refer_to时, 如实返回空值, 不需要自己杜撰或强行拼凑refer_to. cu间不存在refer_to关系是普遍的现象.  

## refer_to关系的含义与注意事项
- refer_to关系的含义是: "某某角色/情况的认定参考某某法律法规文件/法条", "处于某某法律法规文件/法条规定的的特殊情形", "参考附件2", "具体要求见附件4"这种需要参考其他文件/法条来判断的, 属于refer_to; "后续处理应当按照《公司法》、中国证监会和本所的相关规定办理"这种外部的整个法律的遵守也属于refer_to.
- 如果是"应当遵守某某法条", "在某情况下不适用/免于遵守某某法条", "只需要遵守某某法条", 这些在本法律法规文件内部进行遵守/免于遵守的关系, 分别属于should_include, exclude和only_include, 而非refer_to
- 当refer_to的目标为某个具体cu时候直接记录其编号cu_n_k; 当refer_to的目标为本法(或者"本指引")的第a条法条时, 编号为"Law_a". 当refer_to的目标不在本法之间的东西时候, 直接记录改目标为字符串.  
- 示例：
data:
[
    {{"cu_id":"cu_24_1","subject":"大股东 | 一致行动人","condition":"","constraint":"应当共同遵守本指引关于大股东减持股份的规定","contextual_info":""}},
    {{"cu_id":"cu_24_2","subject":"","condition":"","constraint":"","contextual_info":"一致行动人的认定适用《上市公司收购管理办法》规定"}}
]
relation: 
    ("cu_12_2", "refer_to", "一致行动人的认定适用《上市公司收购管理办法》规定")


## 更多经验
 - 先根据关键词进行寻找, 再根据经验和逻辑进行筛选
 - "本指引", "本办法"和"本法"等自指的问题不需要记录"refer_to", 但"参考本指引第n条"这种具体的自指需要(source, refer_to Law_n)
 - refer_to的对象为某个外部法律时, 请直接采用("source_id", "refer_to", "目标法律的字符串"), 不得自己新建Law节点
 - 立法纲领性条款, 例如"依托aaa法, bbb法和ccc法指定本法"的, 不需要构建refer_to关系, 例如"依据《中华人民共和国公司法》《中华人民共和国证券法》《北京证券交易所上市公司持续监管办法（试行）》《上市公司独立董事管理办法》《北京证券交易所股票上市规则（试行）》制定本指引"不需要建立refer_to关系. 
 - 当cu中有"应聘请独立财务顾问寻求帮助"和"应按照证监会的要求"等target_id不为某法律, 某法条或某cu的, 不需要记录refer_to关系. 
 - 一些关键词: 前款, 前述等. 如果遇到"前款"内容被拆分到其他cu, 需要进行refer_to; 如果"前款"涉及的cu特别多, 也可以直接refer_to整个法条Law_n
 - 我们的整个系统另下有其他agent负责管理"only_include"关系, 你不需要处理. 当你看到"主体S_1在情况Cd_1下, **只需要考虑**本法第m, n, k条"这种表述时, 就是典型的"only_include"关系, 不需要添加refer_to关系. 
 - 我们有其他agent负责处理"should_include"关系, 你不需要处理. 当你看到"主体S_1在情况Cd_1下, **应当遵守/应当按照...办理/应当符合**本法第m, n, k条"这种表述时, 就是典型的"only_include"关系. 更多例子: "应当在减持后6个月内继续遵守本指引第四条规定", ""
 - 当且仅当某cu_n_i中提及"请结合本法规第m条第k款"这种表述时, 你可以设立一个"Law_n_k", 并声称("cu_n_i", "refer_to", "Law_n_k"). 你可以这么做的原因是cu与法条的款没有直接对应关系, 为了忠于原文的准确表述可以用符号记录法条的款的信息. 我们会在后处理中解析Law_n_k格式的法条的款的信息, 所以不用担心. 
 - refer_to 有时候可能是一对多的关系, 而且cu的subject, condition, constrain和contextual_info都有可能存在条款引用, 请仔细检查避免遗漏. 


## 识别原则
1. 不修改cu内容，仅建立关联
2. 允许cu之间建立关系, 让cu与法条建立关系, 以及跨法条建立关系
3. 请遵循奥卡姆剃刀原则, 不要增加relation, 除非它是必要的.

## 输出格式
用<RELATIONS>标签包裹的 Python 列表, 列表内为一个个relation元组, 不要有任何注释等赘余内容. 下面是一个输出的样例: 
<RELATIONS>
[
("cu_n_i", "refer_to", "cu_n_j"),
("Law_n", "refer_to", "Law_m")
]
</RELATIONS>


接下来是等待你发掘关系的cu列表:
{cu_list}
"""

## exclude

In [103]:
prompt_get_exclude = """
# cu关系识别指令 exclude（规则排除）

## 角色定位
你是一个资深法律条款引用分析专家，专注识别 cu (合规检查单元) 中的exclude（规则排除）关系

## cu概念简述
cu（ComplianceUnit）是法律条文拆解出的最小合规单元，包含：
- cu_id: cu的编号, 通常为"cu_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东"）
- condition: 触发条件（如"减持股份"） 
- constraint: 约束内容（如"提前15日公告"）
- contextual_info: 补充说明（如价格计算方式）
我们采用cu判断案例的合规性时, 会先检查案例主体是否符合cu主体, 再检查案例中的条件是否符合cu中的条件, 当前两者都满足, 再检查案例中的主体的行为是否违反了cu中的约束. 只有主体, 条件和约束全部满足, 才会认为该案例在该cu上违规, 否则会判定该案例在该cu上不违规. 

## 核心任务
从给定的cu列表中识别 exclude 关系，输出(source_id, exclude, target_id)三元组. 
当target_id有多个时可以用列表承载, 例如(source_id, exclude, [target_id_1, target_id_2])
当所给的cu之间或者cu与法律之间不存在exclude时, 如实返回空值, 不需要自己杜撰或强行拼凑exclude. cu间不存在关系是普遍的现象. 

## exclude关系的含义与注意事项
- exclude关系的含义是: 源cu成立时(源cu的主体subject符合, 且条件condition符合时), 使目标cu失效. 例如, cu_n_i表示主体S_1在情况Cd_1下应当遵守约束Cs_1, 而cu_n_j则声明当主体S_1在情况Cd_2时可以不遵循前款约束Cs_1, 那么就可以理解为cu_n_j对cu_n_i进行了规则排除, 记为("cu_n_j", "exclude", "cu_n_i")
- 当exclude的目标为某个具体cu时候直接记录其编号cu_n_k. 当exclude的目标为本法(或者"本指引")的第n条法条时, 编号为"Law_n". 
- 示例：
data: 
[
    {{"cu_id":"cu_17_1","subject":"上市公司董监高","condition":"在就任时确定的任期内和任期届满后6个月内通过集中竞价、大宗交易、协议转让等方式转让股份且非因司法强制执行、继承、遗赠、依法分割财产等导致股份变动","constraint":"每年转让的股份不得超过其所持本公司股份总数的25%","contextual_info":""}},
    {{"cu_id":"cu_17_2","subject":"上市公司董监高","condition":"所持股份不超过1000股","constraint":"可一次全部转让且不受前款转让比例限制","contextual_info":""}}
]
relation: 
    ("cu_17_2", "exclude", "cu_17_1")

## 更多经验
 - 一般来说只有明确出现"可以不受前款限制", "无需遵守第m, n和k条法律"的才是exclude关系. 
 - 有些cu的condition项存在如"因离婚等原因减持股份, 且不属于证监会规定的除外情况"表述, 这里的"除外"可能导致望文生义, 但其实与我们探讨的exclude没有什么关系. 请你回归定义去理解. 
 - 有时一簇cu存在若干类似的表述, 例如"上市公司, 仅改变募投项目实施地点, 应由董事会审议通过, 免于在股东大会上进行审议"和"上市公司, 改变募集资金用途, 应由董事会和股东大会审议通过", 前者存在"免于..."的表述, 可能导致望文生义, 但是应当注意前者和后者的condition是不同的, 前者仅改变募投项目实施地点, 后者要改变募集资金用途. 
 - 有些cu存在"主体S_1在情况Cd_1下需要遵守限制Cs_1"和"主体S_2在情况Cd_2无需遵守限制Cs_1"的表述, 这里的"无需遵守"可能导致望文生义, 但需注意这里的subject和condition都是不同的, 他们之间也不存在exclude关系. 
 - 如果没有看到"可以不受前款/某某法条限制"等触发词, 请不要自己推理和杜撰exclude关系, 例如公司想进行某项活动, cu_n_i规定需要股东大会审议通过, cu_n_j规定需要董事会审议通过, 这两个要求是并列关系, 不能因为股东大会的等级比董事会高就认定cu_n_i抹除了cu_n_j. 

## 识别原则
1. 不修改cu内容，仅建立关联
2. 允许cu之间建立关系, 让cu与法条建立关系, 以及跨法条建立关系
3. 请遵循奥卡姆剃刀原则, 不要增加relation, 除非它是必要的.

## 输出格式
用<RELATIONS>标签包裹的 Python 列表, 列表内为一个个relation元组, 不要有任何注释等赘余内容. 下面是一个输出的样例: 
<RELATIONS>
[
("cu_n_i", "exclude", "cu_n_j"),
("Law_n", "exclude", "Law_m")
]
</RELATIONS>


接下来是等待你发掘关系的cu列表:
{cu_list}
"""

## only_include

In [104]:
prompt_get_only_include = """
# cu关系识别指令 only_include (仅适用)

## 角色定位
你是一个资深法律条款引用分析专家，专注识别 cu (合规检查单元) 中的only_include (仅适用)关系

## cu概念简述
cu（ComplianceUnit）是法律条文拆解出的最小合规单元，包含：
- cu_id: cu的编号, 通常为"cu_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东"）
- condition: 触发条件（如"减持股份"） 
- constraint: 约束内容（如"提前15日公告"）
- contextual_info: 补充说明（如价格计算方式）
我们采用cu判断案例的合规性时, 会先检查案例主体是否符合cu主体, 再检查案例中的条件是否符合cu中的条件, 当前两者都满足, 再检查案例中的主体的行为是否违反了cu中的约束. 只有主体, 条件和约束全部满足, 才会认为该案例在该cu上违规, 否则会判定该案例在该cu上不违规. 

## 核心任务
从给定的cu列表中识别 only_include 关系，输出(source_id, only_include, target_id)三元组. 
当target_id有多个时可以用列表承载, 例如(source_id, only_include, [target_id_1, target_id_2])
当所给的cu之间或者cu与法律之间不存在only_include时, 如实返回空值, 不需要自己杜撰或强行拼凑only_include. cu间不存在关系是普遍的现象. 

## only_include关系的含义与注意事项
- only_include关系的含义是: 源cu成立时(源cu的主体subject符合, 且条件condition符合时), 只需要考虑目标cu的情况, 而不再需要考虑本法律法规文件内的任何其他的cu. 如果没有排他性, 则不构成only_include关系. 例如, cu_n_i表示主体S_1在情况Cd_1下, 只需要考虑本法第m, n, k条, 这就是视为cu_n_i仅包含其所列示的几条, 记为("cu_n_j", "only_include", ["Law_m", "Law_n", "Law_k"])
- 当only_include的目标为某个具体cu时候直接记录其编号cu_n_k. 当only_include的目标为本法(或者"本指引")的第n条法条时, 编号为"Law_n". 
- 示例：
data:
[
    {{"cu_id":"cu_7_1","subject":"上市公司大股东","condition":"减持通过本所和全国中小企业股份转让系统的竞价或做市交易买入的本公司股份","constraint":"只适用本指引第二条、第三条、第十条、第十一条、第二十三条、第二十七条的规定","contextual_info":""}}
]
relation: 
    ("cu_7_1", "only_include", ["Law_2", "Law_3", "Law_10", "Law_11", "Law_23", "Law_27"])

    
## 更多经验
 - 通常只有当遇到"主体S在情况Cd下仅适用第i, j和k法条(而不再需要适用本法律的任何其他法条)"这种表述出发, 才会触发;only_include. 如果没有排他性, 而是"适用若干条款"的表述, 不是only_include关系. 
 - 在整个工作流程中, 有其他的agent帮忙寻找exclude关系, 这个关系大意是当source的主体和条件触发以后, 可以免除target的考核评估. 你如果发现exclude可以不予理会, 会有其他agent处理这种关系. 
 - 我们有其他的agent帮忙寻找refer_to关系, 当某个cu称"参考某某法律文件"或者"参考本法律法规的第m条进行认定", 就是refer_to关系. 你不需要处理refer_to关系. 
 - 我们有其他的agent负责寻找should_include关系, 这个关系是指当且仅当某cu明确声明强制纳入本法律法规内哪些法条的, 例如"应当符合/遵循/参照/按照本指引第x, ..., z条的规定...". 这个关系有应当遵守的意思, 但并非是"仅遵守". 你不需要处理should_include关系. should_include关系是没有排他性的, 而你需要负责的only_include是有排他性的, 也即不再需要遵守目标cu以外的其他任何cu. 
 - only_include是一个很少见的关系. 当你觉得你发现了only_include你需要仔细想想: 这个cu的意思真是可以排除本法条的其他一切吗? 


## 识别原则
1. 不修改cu内容，仅建立关联
2. 允许cu之间建立关系, 让cu与法条建立关系, 以及跨法条建立关系
3. 请遵循奥卡姆剃刀原则, 不要增加relation, 除非它是必要的.

## 输出格式
用<RELATIONS>标签包裹的 Python 列表, 列表内为一个个relation元组, 不要有任何注释等赘余内容. 下面是一个输出的样例: 
<RELATIONS>
[
("cu_n_i", "only_include", "cu_n_j"),
("Law_n", "only_include", "Law_m")
]
</RELATIONS>


接下来是等待你发掘关系的cu列表:
{cu_list}
"""

## should_include

In [105]:
prompt_get_should_include = """
# cu关系识别指令 should_include (强制纳入)

## 角色定位
你是一个资深法律条款引用分析专家，专注识别 cu (合规检查单元) 中的should_include (强制纳入)关系

## cu概念简述
cu（ComplianceUnit）是法律条文拆解出的最小合规单元，包含：
- cu_id: cu的编号, 通常为"cu_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东"）
- condition: 触发条件（如"减持股份"） 
- constraint: 约束内容（如"提前15日公告"）
- contextual_info: 补充说明（如价格计算方式）
我们采用cu判断案例的合规性时, 会先检查案例主体是否符合cu主体, 再检查案例中的条件是否符合cu中的条件, 当前两者都满足, 再检查案例中的主体的行为是否违反了cu中的约束. 只有主体, 条件和约束全部满足, 才会认为该案例在该cu上违规, 否则会判定该案例在该cu上不违规. 

## 核心任务
从给定的cu列表中识别 should_include 关系，输出(source_id, should_include, target_id)三元组. 
当target_id有多个时可以用列表承载, 例如(source_id, should_include, [target_id_1, target_id_2])
当所给的cu之间或者cu与法律之间不存在should_include时, 如实返回空值, 不需要自己杜撰或强行拼凑should_include. cu间不存在关系是普遍的现象. 

## should_include关系的含义与注意事项
- should_include关系的定义: 当cu_n_k中出现 {{"应当符合/遵循本指引第m, n, k条的要求", "应当参照/按照/参见本指引第m, n, k条处理",}} 时, 就是当前的cu_n_k对Law_x, ..., Law_z存在should_include关系. 
- 只有明确声明强制纳入本法律法规内哪些法条的, 才是should_include关系. 指向目标不在本法律法规文件内部的, 以及未明确说明的强制纳入哪些法条的不予考虑. 例如"一致行动人的认定参照本所上市准则", 其莫表不再本法律文件(本法/本指引)之内, 而且这是一个refer_to关系, 你不需要考虑. 
- 示例：
data:
[
    {{"cu_id":"cu_50_1","subject":"上市公司","condition":"实施要约回购","constraint":"应当符合本指引第十三条至第十五条、第十七条、第十九条、第二十条的规定","contextual_info":""}}
]
relation: 
    ("cu_7_1", "should_include", ["Law_13", "Law_14", "Law_15", "Law_17", "Law_29", "Law_20"])

    
## 更多经验
 - 我们有另外的agent处理refer_to关系, 这与你负责的should_include容易混淆. refer_to关系是指, 源cu需要结合目标条款才能完整解释. 常见的refer_to表述有: "应当共同遵守本指引关于大股东减持股份的规定", "一致行动人的认定适用《上市公司收购管理办法》规定". 这几种情况没有明确声明强制纳入本法律法规内哪些法条的规定, 因此你都不需要考虑建立should_include关系. 
 - 我们有另外的agent负责管理"only_include"关系, 你不需要处理. 当你看到"主体S_1在情况Cd_1下, 只需要考虑本法第m, n, k条"这种表述时, 就是典型的"only_include"关系, 你不需要再在这些cu之间添加should_include关系. 
 - 我们有其他的agent帮忙寻找refer_to关系, 当某个cu称"参考某某法律文件"或者"参考本法律法规的第m条进行认定", 就是refer_to关系. 你不需要处理refer_to关系. 你只需要处理明确的""


## 识别原则
1. 不修改cu内容，仅建立关联
2. 允许cu之间建立关系, 让cu与法条建立关系, 以及跨法条建立关系
3. 请遵循奥卡姆剃刀原则, 不要增加relation, 除非它是必要的.

## 输出格式
用<RELATIONS>标签包裹的 Python 列表, 列表内为一个个relation元组, 不要有任何注释等赘余内容. 下面是一个输出的样例: 
<RELATIONS>
[
("cu_n_i", "should_include", "cu_n_j"),
("Law_n", "should_include", "Law_m")
]
</RELATIONS>


接下来是等待你发掘关系的cu列表:
{cu_list}
"""

# 批量调用LLM获取回复

In [106]:
prompt_list = [
    prompt_get_refer_to,
    # prompt_get_combine,
    # prompt_get_depend,
    prompt_get_exclude,
    prompt_get_only_include,
    prompt_get_should_include
]


## 异步获取回复

In [107]:
import asyncio
import pandas as pd
from tqdm.asyncio import tqdm_asyncio
from utils.call_gpt import call_gpt_async
import csv
import os

async def process_cu_relations_async(
    file_name, 
    prompt_list,
    max_concurrency=10,
):
    """
    异步处理函数
    
    参数：
    prompt_list: 提示词模板列表
    cu_list: 用于格式化的cu列表
    max_concurrency: 最大并行请求数（默认10）
    """

    input_dir = r"law_to_ComplianceUnit/st_2_ComplianceUnit/GT" 
    output_dir=r"law_to_ComplianceUnit/st_3_ComplianceUnit_relations/raw_response"
    os.makedirs(output_dir, exist_ok=True)

    input_path = os.path.join(input_dir, file_name)
    output_path = os.path.join(output_dir, file_name)
    columns_to_extract = ["cu_id", "subject", "condition", "constraint", "contextual_info"]

    # 读取CSV文件
    df = pd.read_excel(input_path, engine='openpyxl')
    extracted_df = df[columns_to_extract]
    cu_list = extracted_df.to_dict(orient="records")

    # 共享状态
    results = []
    failed_prompts = []
    semaphore = asyncio.Semaphore(max_concurrency)

    async def process_single(prompt_template):
        """处理单个提示词的异步任务"""
        async with semaphore:
            try:
                formatted_prompt = prompt_template.format(cu_list=cu_list)
                
                content, reasoning_content, api_usage = await call_gpt_async(
                    prompt=formatted_prompt,
                    api_key=your_api_key,
                    base_url=your_api_base,
                    model=your_model_name,
                    # temperature=0.6,
                )

                return {
                    "response": content,
                    "reasoning_content": reasoning_content,
                    "api_usage": api_usage
                }
            except Exception as e:
                failed_prompts.append(str(prompt_template))
                return None

    # 创建所有任务
    tasks = [process_single(prompt) for prompt in prompt_list]
    
    # 执行并带进度条
    pbar = tqdm_asyncio(total=len(tasks), desc="Processing prompts")
    
    for future in asyncio.as_completed(tasks):
        result = await future
        if result:
            results.append(result)
        pbar.update()

    pbar.close()

    # 保存结果到单个CSV文件
    with open(output_path, "w", encoding='utf-8-sig', newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["response", "reasoning_content", "api_usage"])
        writer.writeheader()
        writer.writerows(results)

    # 打印简要报告
    print(f"\nSuccess: {len(results)}")
    print(f"Failed: {len(failed_prompts)}")

    return failed_prompts

In [108]:
# failed_prompts = await process_cu_relations_async(
#     prompt_list=[
#         prompt_get_refer_to,
#         prompt_get_combine,
#         prompt_get_depend,
#         prompt_get_exclude,
#         prompt_get_only_include,
#     ],
#     file_name = '北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理', 
#     max_concurrency=5
# )

## 解析回复结果

In [117]:
import csv
import re
import json
import os

def get_relations_from_responses(file_name):

    input_dir = r"law_to_ComplianceUnit/st_3_ComplianceUnit_relations/raw_response"
    output_dir = r"law_to_ComplianceUnit/st_3_ComplianceUnit_relations"
    os.makedirs(output_dir, exist_ok=True)

    input_path = os.path.join(input_dir, file_name)
    output_path = os.path.join(output_dir, file_name)

    def clean_response(response):
        """清理和解析response列中的关系数据"""
        try:
            # 统一处理转义符号
            response = response.replace("<\\RELATIONS>", "</RELATIONS>")
            response = response.replace("<\\\\RELATIONS>", "</RELATIONS>")
            
            # 提取所有RELATIONS标签内容
            matches = re.findall(r'<RELATIONS>(.*?)</RELATIONS>', response, re.DOTALL)
            
            if matches:
                # 选择内容最长的匹配项
                longest_content = max(matches, key=lambda x: len(x.strip())).strip()
                
                # 数据格式转换
                longest_content = longest_content.replace('""', '"')  # 处理双引号转义
                longest_content = longest_content.replace('(', '[').replace(')', ']')  # 转换括号格式
                
                # 解析为JSON数组
                return json.loads(longest_content)
            return []
        except json.JSONDecodeError as e:
            print(f"JSON解析错误: {e}")
            print(f"问题内容: {longest_content}")
            return []
        except Exception as e:
            print(f"数据处理异常: {e}")
            return []

    # 读取原始CSV文件
    with open(input_path, mode='r', encoding='utf-8-sig') as infile:
        reader = csv.DictReader(infile)
        law_articles = list(reader)

    # 处理关系数据
    split_data = []
    fieldnames = ['source', 'relation', 'target']
    # print(law_articles[0])
    
    for row in law_articles:
        relations = clean_response(row['response'])
        for rel_entry in relations:
            # 校验数据格式
            if not isinstance(rel_entry, list) or len(rel_entry) != 3:
                print(f"异常条目: {rel_entry}")
                continue
            
            source, relation, targets = rel_entry
            # 处理target为列表的情况
            if isinstance(targets, list):
                for target in targets:
                    split_data.append({
                        'source': source.strip('"'),  # 去除可能的残留引号
                        'relation': relation.strip('"'),
                        'target': target.strip('"')
                    })
            else:
                split_data.append({
                    'source': source.strip('"'),
                    'relation': relation.strip('"'),
                    'target': targets.strip('"')
                })

    # 写入处理后的CSV文件
    output_path = output_path.replace('.xlsx', '.csv')
    with open(output_path, mode='w', encoding='utf-8-sig', newline='') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(split_data)
    
    print(f'文件已保存至: {output_path}')

In [110]:
# 使用示例
# get_relations_from_responses("北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.csv")

## 主workflow

In [118]:
prompt_list=[
        prompt_get_refer_to,
        # prompt_get_combine,
        # prompt_get_depend,
        prompt_get_exclude,
        prompt_get_only_include,
        prompt_get_should_include,
    ]

# 定义文件目录路径
directory_path = r"law_to_ComplianceUnit/st_2_ComplianceUnit/GT"

filenames = [
    f for f in os.listdir(directory_path) if f.endswith('.xlsx')
]

filenames 

['北京证券交易所上市公司持续监管指引第5号——要约收购.xlsx',
 '北京证券交易所上市公司持续监管指引第6号——内幕信息知情人管理及报送.xlsx',
 '北京证券交易所上市公司持续监管指引第9号——募集资金管理.xlsx',
 '北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.xlsx',
 '北京证券交易所上市公司持续监管指引第10号——权益分派.xlsx',
 '北京证券交易所上市公司持续监管指引第4号——股份回购.xlsx',
 '北京证券交易所上市公司持续监管指引第3号——股权激励和员工持股计划.xlsx',
 '北京证券交易所上市公司持续监管指引第7号——转板.xlsx',
 '北京证券交易所上市公司持续监管指引第2号——季度报告.xlsx',
 '北京证券交易所上市公司持续监管指引第1号——独立董事.xlsx']

In [ ]:

for filename in filenames:
    failed_prompts = await process_cu_relations_async(
    prompt_list=prompt_list,
    file_name = filename, 
    max_concurrency=10
    )
    get_relations_from_responses(filename)



文件已保存至: law_to_ComplianceUnit/st_3_ComplianceUnit_relations/北京证券交易所上市公司持续监管指引第5号——要约收购.csv
文件已保存至: law_to_ComplianceUnit/st_3_ComplianceUnit_relations/北京证券交易所上市公司持续监管指引第6号——内幕信息知情人管理及报送.csv
文件已保存至: law_to_ComplianceUnit/st_3_ComplianceUnit_relations/北京证券交易所上市公司持续监管指引第9号——募集资金管理.csv
文件已保存至: law_to_ComplianceUnit/st_3_ComplianceUnit_relations/北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.csv
文件已保存至: law_to_ComplianceUnit/st_3_ComplianceUnit_relations/北京证券交易所上市公司持续监管指引第10号——权益分派.csv
文件已保存至: law_to_ComplianceUnit/st_3_ComplianceUnit_relations/北京证券交易所上市公司持续监管指引第4号——股份回购.csv
文件已保存至: law_to_ComplianceUnit/st_3_ComplianceUnit_relations/北京证券交易所上市公司持续监管指引第3号——股权激励和员工持股计划.csv
文件已保存至: law_to_ComplianceUnit/st_3_ComplianceUnit_relations/北京证券交易所上市公司持续监管指引第7号——转板.csv
文件已保存至: law_to_ComplianceUnit/st_3_ComplianceUnit_relations/北京证券交易所上市公司持续监管指引第2号——季度报告.csv
文件已保存至: law_to_ComplianceUnit/st_3_ComplianceUnit_relations/北京证券交易所上市公司持续监管指引第1号——独立董事.csv
